In [1]:
import json
import pprint as pp
import os
from pathlib import Path
from dotenv import load_dotenv
import boto3
from botocore.exceptions import ClientError

# Section 1 - Environment variables loading

In [2]:
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

aws_account_id = os.getenv('aws_account_id')
aws_sdk_access_id = os.getenv('aws_sdk_access_id')
aws_sdk_access_secret = os.getenv('aws_sdk_access_secret')
aws_worker_access_id = os.getenv('aws_worker_access_id')
aws_worker_access_secret = os.getenv('aws_worker_access_secret')

aws_region = os.getenv('aws_region')
aws_private_bucket = os.getenv('aws_private_bucket')
aws_deploy_bucket = os.getenv('aws_deploy_bucket')

bing_api_key = os.getenv('bing_api_key')

task_name = os.getenv('task_name')
batch_name = os.getenv('batch_name')

# print("Printing env vars values:")
# print(f"AWS ACCOUNT ID: {aws_account_id}")
# print(f"AWS SDK ACCESS ID: {aws_sdk_access_id}")
# print(f"AWS SDK ACCESS SECRET: {aws_sdk_access_secret}")
# print(f"AWS WORKER ACCESS ID: {aws_worker_access_id}")
# print(f"AWS WORKER ACCESS SECRET: {aws_worker_access_secret}")
# print(f"AWS REGION: {aws_region}")
# print(f"AWS PRIVATE BUCKET: {aws_private_bucket}")
# print(f"AWS DEPLOY BUCKET: {aws_deploy_bucket}")
# print(f"BING API KEY: {bing_api_key}")
# print(f"TASK NAME: {task_name}")
# print(f"BATCH NAME: {batch_name}")

# Section 2 - Private and deploy bucket creation

In [46]:
s3_client = boto3.client('s3', aws_access_key_id=aws_sdk_access_id, aws_secret_access_key=aws_sdk_access_secret)
s3_resource = boto3.resource('s3')

buckets = []
for bucket in s3_resource.buckets.all():
    buckets.append(bucket.name)

print(f"---------- INITIALIZING PRIVATE BUCKET {aws_private_bucket} ----------")

try:
    response = s3_client.create_bucket(Bucket=aws_private_bucket, CreateBucketConfiguration={'LocationConstraint': aws_region})
    print(f"Bucket creation successfully completed, HTTP STATUS CODE: {response['ResponseMetadata']['HTTPStatusCode']}.")
    pp.pprint(response)
except ClientError as error:
    print(f"{error.response['Error']['Message']}, HTTP STATUS CODE: {error.response['ResponseMetadata']['HTTPStatusCode']}.")

response = s3_client.put_public_access_block(
    Bucket=aws_private_bucket,
    PublicAccessBlockConfiguration={
        'BlockPublicAcls': True,
        'IgnorePublicAcls': True,
        'BlockPublicPolicy': True,
        'RestrictPublicBuckets': True
    },
)
print(f"Public access blocked, HTTP STATUS CODE: {response['ResponseMetadata']['HTTPStatusCode']}.")

private_bucket_policy = {
    "Version": "2012-10-17",
    "Id": "private-bucket-policy",
    "Statement": [
        {
            "Sid": "allow-bucket-interaction",
            "Effect": "Allow",
            "Principal": {
                "AWS": f"arn:aws:iam::{aws_account_id}:user/worker"
            },
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{aws_private_bucket}",
                f"arn:aws:s3:::{aws_private_bucket}/*"
            ]
        }
    ]
}

response = s3_client.put_bucket_policy(Bucket=aws_private_bucket, Policy=json.dumps(private_bucket_policy))
print(f"Bucket policy successfully set up, HTTP STATUS CODE: {response['ResponseMetadata']['HTTPStatusCode']}.")

cors_configuration = {
    'CORSRules': [{
        'AllowedHeaders': ['*'],
        'AllowedMethods': ['GET', 'HEAD', 'PUT'],
        'AllowedOrigins': ['*'],
        'ExposeHeaders': [],
        'MaxAgeSeconds': 3000
    }]
}

response = s3_client.get_bucket_cors(Bucket=aws_private_bucket)
s3_client.put_bucket_cors(Bucket=aws_private_bucket, CORSConfiguration=cors_configuration)
print(f"CORS Configuration successfully set up, HTTP STATUS CODE: {response['ResponseMetadata']['HTTPStatusCode']}.")

print(f"---------- INITIALIZATION COMPLETED ----------")


# if aws_deploy_bucket not in buckets:
#     print(f"---------- INITIALIZING DEPLOY BUCKET {aws_deploy_bucket} ----------")
#     deploy_bucket_check = create_bucket(aws_deploy_bucket, aws_region)
#     if deploy_bucket_check:
#         print(f"Creation successfully completed")
#     print(f"---------- INITIALIZATION COMPLETED ----------")
# else:
#     print(f"Deploy bucket {aws_deploy_bucket} already initialized")

---------- INITIALIZING PRIVATE BUCKET private-bucket-test-fdssfsdfdssdds ----------
Your previous request to create the named bucket succeeded and you already own it., HTTP STATUS CODE: 409.
Public access blocked, HTTP STATUS CODE: 200.
Bucket policy successfully set up, HTTP STATUS CODE: 204.
CORS Configuration successfully set up, HTTP STATUS CODE: 200.
---------- INITIALIZATION COMPLETED ----------
